In [1]:
%pylab inline
import scipy.stats as stats
import pandas as pd
import seaborn as sb
rcParams['font.size']=12
rcParams['pdf.fonttype']=42


Populating the interactive namespace from numpy and matplotlib


In [2]:
# load replicates map
replicatemap = pd.read_csv('replicate_map.csv',index_col=0,header=0)

In [3]:
# pDNA -> fill out cell ID
replicatemap['DepMap_ID'].fillna('pDNA',inplace=True)

In [4]:
replicatemap.head(3)

,DepMap_ID,pDNA_batch
replicate_ID,,
MIA Paca-2-311Cas9 Rep A p6_batch0,ACH-000601,0
MIA Paca-2-311Cas9 Rep B p6_batch0,ACH-000601,0
MIA Paca-2-311Cas9 Rep C p6_batch0,ACH-000601,0


In [5]:
# uniquce cell lines
len(replicatemap['DepMap_ID'].unique())

564

In [6]:
# pDNA batches
replicatemap['pDNA_batch'].unique()

array([0, 1, 2, 3])

In [7]:
# map batch, controls, samples
controls = dict()  # [batch] list[repid]
samples = dict()  # [batch][cell] list[repid]
for repid in replicatemap.index:
    cell = replicatemap.get_value(repid,'DepMap_ID')
    batch = replicatemap.get_value(repid,'pDNA_batch')
    if cell == 'pDNA':
        if batch not in controls:
            controls[batch] = list()
        controls[batch].append(repid)
    else:
        if batch not in samples:
            samples[batch] = dict()
        if cell not in samples[batch]:
            samples[batch][cell] = list()
        samples[batch][cell].append(repid)
    

/home/ekim8/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  """
/home/ekim8/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: get_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [8]:
controls

{1: ['pDNA Avana4_010115_1.5Ex_batch1',
  'pDNA Avana4_060115_1.5Ex_batch1',
  'pDNA Avana4_0101215_0.55Ex_batch1',
  'pDNA Avana4_060115_0.55Ex_batch1'],
 0: ['pDNA Avana4_010115_1.5Ex_batch0', 'pDNA Avana4_060115_1.5Ex_batch0'],
 3: ['Avana 4+ Hu pDNA (M-AA40, 9/30/15)_batch3',
  'Avana 4+ Hu pDNA (M-AA40, 9/30/15) (0.2pg/uL)_batch3',
  'Avana 4+ Hu pDNA (M-AA40, 9/30/15) 0.2pg/uL_batch3',
  'Avana 4+ Hu pDNA (M-AF34, 11/27/18) 0.2pg/uL_batch3'],
 2: ['Avana4pDNA20160601-311cas9 RepG09_batch2',
  'Avana4pDNA20160601-311cas9 RepG10_batch2',
  'Avana4pDNA20160601-311cas9 RepG11_batch2',
  'Avana4pDNA20160601-311cas9 RepG12_batch2']}

In [9]:
# load DepMap 19Q2 readcount file
readcount = pd.read_csv('raw_readcounts.csv',index_col=0,header=0)

In [10]:
readcount.head(3)

,143B-311Cas9_RepA_p6_batch3,2313287-311Cas9_RepA_p5_batch3,2313287-311Cas9_RepB_p5_batch3,253J-311Cas9_RepA_p5_batch3,42-MG-BA-311Cas9_RepA_p6_batch3,42-MG-BA-311Cas9_RepB_p6_batch3,5637-311Cas9_RepA_p6_batch3,5637-311Cas9_RepB_p6_batch3,59M-311Cas9_RepA_p4_batch3,59M-311Cas9_RepB_p4_batch3,...,YKG1-311Cas9_RepA_p6_batch3,YKG1-311Cas9_RepB_p6_batch3,ZR-75-1-311Cas9_RepA_p5_batch2,ZR-75-1-311Cas9_RepB_p5_batch2,pDNA Avana4_010115_1.5Ex_batch0,pDNA Avana4_010115_1.5Ex_batch1,pDNA Avana4_0101215_0.55Ex_batch1,pDNA Avana4_060115_0.55Ex_batch1,pDNA Avana4_060115_1.5Ex_batch0,pDNA Avana4_060115_1.5Ex_batch1
AAAAAAATCCAGCAATGCAG,833.0,327.0,188.0,546.0,213.0,349.0,280.0,489.0,683.0,373.0,...,565.0,556.0,226.0,386.0,305.0,300.0,190.0,176.0,324.0,289.0
AAAAAACCCGTAGATAGCCT,1089.0,383.0,391.0,1151.0,736.0,727.0,565.0,770.0,1279.0,720.0,...,637.0,773.0,632.0,673.0,398.0,396.0,254.0,230.0,421.0,445.0
AAAAAAGAAGAAAAAACCAG,95.0,32.0,69.0,72.0,46.0,60.0,99.0,102.0,18.0,76.0,...,171.0,57.0,44.0,39.0,93.0,75.0,42.0,31.0,74.0,69.0


In [11]:
# load CCDS data to decide protein-coding genes
ccdsdata = pd.read_table('CCDS.20180614.txt',index_col=None,header=0)
ccdsdata = ccdsdata[(ccdsdata['ccds_status'] == 'Public') | (ccdsdata['ccds_status'] == 'Reviewed, update pending')]

In [12]:
ccdsdata.head(3)

,#chromosome,nc_accession,gene,gene_id,ccds_id,ccds_status,cds_strand,cds_from,cds_to,cds_locations,match_type
1,1,NC_000001.11,SAMD11,148398,CCDS2.2,Public,+,925941,944152,"[925941-926012, 930154-930335, 931038-931088, ...",Identical
2,1,NC_000001.11,NOC2L,26155,CCDS3.1,Public,-,944693,959239,"[944693-944799, 945056-945145, 945517-945652, ...",Identical
3,1,NC_000001.11,PLEKHN1,84069,CCDS4.1,Public,+,966531,974574,"[966531-966613, 966703-966802, 970276-970422, ...",Identical


In [13]:
gene2id_ccds = dict()
id2gene_ccds = dict()

for i in ccdsdata.index:
    if ccdsdata.loc[i]['gene'] in gene2id_ccds and gene2id_ccds[ccdsdata.loc[i]['gene']] != ccdsdata.loc[i]['gene_id']:
        print( i) # check dup
    gene2id_ccds[ccdsdata.loc[i]['gene']] = ccdsdata.loc[i]['gene_id']
    id2gene_ccds[ccdsdata.loc[i]['gene_id']] = ccdsdata.loc[i]['gene']
    

In [14]:
#map exon for crisprcleanR
gene2cds = dict()
gene2cdslength = dict()
for i in ccdsdata.index:
    if ccdsdata.loc[i]['match_type'] == 'Partial':  # no info so skip
        continue
    if ccdsdata.loc[i]['gene'] not in gene2cds or abs(int(ccdsdata.loc[i]['cds_from'])-int(ccdsdata.loc[i]['cds_to']) ) > gene2cdslength[ccdsdata.loc[i]['gene']]:
        # longer gene if same gene
        gene2cds[ccdsdata.loc[i]['gene']] = ccdsdata.loc[i]['cds_locations'].replace("[","").replace("]","").replace(" ","").split(",")
        gene2cdslength[ccdsdata.loc[i]['gene']] = abs(int(ccdsdata.loc[i]['cds_from'])-int(ccdsdata.loc[i]['cds_to']) )
        

/home/ekim8/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  """
/home/ekim8/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  import sys
/home/ekim8/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':
/home/ekim8/anaconda3/lib/pytho

In [15]:
guidegenemap = pd.read_csv('guide_map.csv',index_col=0,header=0)

sgrna2gene = dict()
sgrnacount = dict()
old2new = dict()
finalgenes = set()
for i in range(len(guidegenemap.index)):
    sgrna = guidegenemap.index[i]
    temp = guidegenemap.iloc[i]['gene'].replace(")","").split(" (")
    gene = temp[0]
    entrezid = int(temp[1])
    if entrezid in id2gene_ccds:
        old2new[gene]=gene
        if id2gene_ccds[entrezid] != gene:
            old2new[gene]=id2gene_ccds[entrezid]
            gene = id2gene_ccds[entrezid]
        finalgenes.add(gene)
        sgrna2gene[sgrna] = gene
        if sgrna not in sgrnacount:
            sgrnacount[sgrna]=0
        sgrnacount[sgrna]+=1

#select sgRNA target only one protein-coding gene
targetsgrna = dict()
for sgrna in sgrnacount:
    if sgrnacount[sgrna]==1:
        targetsgrna[sgrna] = sgrna2gene[sgrna]

/home/ekim8/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  if __name__ == '__main__':


In [16]:
len(finalgenes)

17626

In [17]:
guidegenemap.head(10)

,genome_alignment,gene,n_alignments
sgrna,,,
AAAAAAATCCAGCAATGCAG,chr10_112724378_+,SHOC2 (8036),1
AAAAAACCCGTAGATAGCCT,chr12_95397391_+,NDUFA12 (55967),1
AAAAAAGAAGAAAAAACCAG,chr4_76891509_-,SDAD1 (55153),1
AAAAAAGCTCAAGAAGGAGG,chr2_33813513_-,FAM98A (25940),1
AAAAAAGGCTGTAAAAGCGT,chr19_20002409_+,ZNF253 (56242),1
AAAAAAGGGCTCCAAAAAGG,chr6_26199835_+,HIST1H2BF (8343),1
AAAAACAACACATCAGAGCG,chr14_64688272_-,SYNE2 (23224),1
AAAAACTCTGGGAAATGACT,chr1_229440884_+,SPHAR (10638),1
AAAAAGACAACCTCGCCCTG,chr11_64757251_-,BATF2 (116071),1


In [21]:
temp = guidegenemap.loc[targetsgrna]
tempdf = dict()
for i in range(len(temp.index)):
    gene = temp.iloc[i]['gene'].replace(")","").split(" (")[0]
    entrezid = temp.iloc[i]['gene'].replace(")","").split(" (")[1]
    if gene in old2new and old2new[gene] in finalgenes:
        temprow = [ temp.iloc[i]['genome_alignment'] , old2new[gene], entrezid, temp.iloc[i]['n_alignments'] ]
        tempdf[temp.index[i]] = temprow
        
        
tempdf = pd.DataFrame.from_dict(tempdf).T.rename(columns={0:"genome_alignment",1:"gene",2:"entrezid",3:"n_alignments"})


In [22]:
print (" ".join(setdiff1d(list(finalgenes),tempdf['gene'].unique())))

AGAP4 AMY1A AMY1B AMY1C ANKRD20A1 ANKRD20A2 ANKRD20A3 ANKRD20A4 ARL17A ARL17B BOLA2 BOLA2B C4A C4B CBWD3 CCZ1 CCZ1B CGB1 CGB2 CGB3 CGB5 CGB8 CKMT1A CKMT1B CSH1 CSNK2A3 CTAGE4 CTAGE6 CTAGE8 DDT DDTL DEFA1 DEFA1B DEFA3 DEFB103A DEFB103B DEFB104A DEFB104B DEFB105A DEFB105B DEFB106A DEFB106B DEFB107A DEFB107B DEFB130A DEFB130B EIF3C ELOA3 ELOA3B FAM25C FAM25G FAM72A FAM72B FAM72D FAM86B1 FCGR1A FCGR1B FOXD4L3 FOXD4L6 FRG2 GCOM1 GOLGA6C GOLGA6L9 GOLGA8A GOLGA8B GOLGA8J GOLGA8K GOLGA8O GPR89A GPR89B GRAP GRAPL GTF2IRD2 GTF2IRD2B GYPB HBA1 HBA2 HIST1H4J HIST1H4K HIST2H2AA3 HIST2H2AA4 HIST2H3A HIST2H3C HIST2H4A HIST2H4B IFNA13 KRTAP2-1 KRTAP2-2 KRTAP2-3 KRTAP2-4 KRTAP4-7 KRTAP9-2 KRTAP9-4 KRTAP9-7 LCE1C LCE1F LCE2C LIMS3 LIMS4 LYZL1 LYZL2 MBD3L3 MBD3L4 MBD3L5 MYZAP NBPF15 NBPF4 NBPF6 NOMO3 NPIPA1 NPIPA2 NPIPA3 NPIPA7 NPIPA8 OBP2A OBP2B OR11H12 OR2A1 OR2A42 OR4F16 OR4F17 OR4F29 OR4F3 OR4F4 PGA3 PGA4 PGA5 PLGLB1 PLGLB2 POLR2J POLR2J2 POLR2J3 POTEB POTEB2 POTEJ PPIAL4A PPIAL4C PRAMEF15 PRAMEF5 PR

In [23]:
# save modified guide gene map
tempdf.to_csv('guide_gene_map_mod.csv')

In [25]:
#add gene symbol to readcount
intst = intersect1d(readcount.index,list(targetsgrna.keys()))
readcount = readcount.loc[intst]
readcount['GENE'] = pd.Series(targetsgrna)

In [26]:
readcount

,143B-311Cas9_RepA_p6_batch3,2313287-311Cas9_RepA_p5_batch3,2313287-311Cas9_RepB_p5_batch3,253J-311Cas9_RepA_p5_batch3,42-MG-BA-311Cas9_RepA_p6_batch3,42-MG-BA-311Cas9_RepB_p6_batch3,5637-311Cas9_RepA_p6_batch3,5637-311Cas9_RepB_p6_batch3,59M-311Cas9_RepA_p4_batch3,59M-311Cas9_RepB_p4_batch3,...,YKG1-311Cas9_RepB_p6_batch3,ZR-75-1-311Cas9_RepA_p5_batch2,ZR-75-1-311Cas9_RepB_p5_batch2,pDNA Avana4_010115_1.5Ex_batch0,pDNA Avana4_010115_1.5Ex_batch1,pDNA Avana4_0101215_0.55Ex_batch1,pDNA Avana4_060115_0.55Ex_batch1,pDNA Avana4_060115_1.5Ex_batch0,pDNA Avana4_060115_1.5Ex_batch1,GENE
AAAAAAATCCAGCAATGCAG,833.0,327.0,188.0,546.0,213.0,349.0,280.0,489.0,683.0,373.0,...,556.0,226.0,386.0,305.0,300.0,190.0,176.0,324.0,289.0,SHOC2
AAAAAACCCGTAGATAGCCT,1089.0,383.0,391.0,1151.0,736.0,727.0,565.0,770.0,1279.0,720.0,...,773.0,632.0,673.0,398.0,396.0,254.0,230.0,421.0,445.0,NDUFA12
AAAAAAGAAGAAAAAACCAG,95.0,32.0,69.0,72.0,46.0,60.0,99.0,102.0,18.0,76.0,...,57.0,44.0,39.0,93.0,75.0,42.0,31.0,74.0,69.0,SDAD1
AAAAAAGCTCAAGAAGGAGG,236.0,197.0,59.0,438.0,152.0,252.0,138.0,161.0,378.0,138.0,...,236.0,454.0,214.0,131.0,120.0,68.0,90.0,135.0,135.0,FAM98A
AAAAAAGGCTGTAAAAGCGT,460.0,117.0,125.0,196.0,251.0,204.0,208.0,338.0,254.0,85.0,...,186.0,301.0,262.0,174.0,153.0,109.0,118.0,174.0,193.0,ZNF253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTTGGAGAGATGTACGA,334.0,120.0,266.0,293.0,301.0,302.0,486.0,369.0,473.0,577.0,...,276.0,314.0,392.0,182.0,199.0,140.0,118.0,172.0,141.0,LIPH
TTTGTTGGCACAAATACGGG,1112.0,590.0,469.0,888.0,963.0,850.0,1005.0,981.0,1202.0,909.0,...,1062.0,1182.0,1285.0,513.0,503.0,356.0,289.0,486.0,486.0,RPL10L
TTTGTTTCCTCTGAAGTAGT,116.0,371.0,381.0,99.0,217.0,535.0,523.0,283.0,118.0,117.0,...,447.0,301.0,128.0,886.0,792.0,530.0,516.0,762.0,771.0,RPS24
TTTGTTTCCTCTTCTCGAGG,1065.0,398.0,345.0,1011.0,640.0,511.0,565.0,692.0,301.0,1439.0,...,836.0,481.0,803.0,281.0,259.0,158.0,173.0,268.0,236.0,CRISPLD1


In [43]:
# file name info
files = pd.DataFrame(columns=['foldchangefile','col','bffile','prfile','batch'])

for batch in samples:
    cols = ['GENE']
    cell2col = dict()
    for cell in samples[batch]:
        cell2col[cell] = list()
        for repid in samples[batch][cell]:
            cell2col[cell].append(len(cols))
            cols.append(repid)
    cont2col = list()
    for repid in controls[batch]:   # to avoid column number change after calculating fc, control is the last
        cont2col.append(len(cols))
        cols.append(repid)
    
    readcount_batch_file = 'Avana19Q2_readcount.batch'+str(batch)
    readcount[cols].index.rename("sgRNA",inplace=True)
    readcount[cols].to_csv(readcount_batch_file,sep='\t')
    #print( "./BAGEL_110.py fc -i " + readcount_batch_file + " -o " + readcount_batch_file + " -c " + ",".join(map(str,cont2col)))
    fcfile = readcount_batch_file + ".foldchange"
    for cell in cell2col:
        bffile =  fcfile + "." + cell + ".bf"
        prfile =  fcfile + "." + cell + ".pr"
        files.loc[cell] = [fcfile,",".join(map(str,cell2col[cell])), bffile, prfile, batch]
        #print( "./BAGEL_110.py bf -i " + fcfile + " -o " + bffile + " -c " + ",".join(map(str,cell2col[cell])) + " -e training_ess -n training_noness")
        #print( "./BAGEL_110.py pr -i " + bffile + " -o " + prfile + " -e training_ess -n training_noness")
    #print()
    #print()
    #print()

In [28]:
files

,foldchangefile,col,bffile,prfile,batch
ACH-000601,Avana19Q2_readcount.batch0.foldchange,"1,2,3,4",Avana19Q2_readcount.batch0.foldchange.ACH-0006...,Avana19Q2_readcount.batch0.foldchange.ACH-0006...,0
ACH-000496,Avana19Q2_readcount.batch0.foldchange,"5,6,7,8",Avana19Q2_readcount.batch0.foldchange.ACH-0004...,Avana19Q2_readcount.batch0.foldchange.ACH-0004...,0
ACH-000222,Avana19Q2_readcount.batch0.foldchange,"9,10,11,12",Avana19Q2_readcount.batch0.foldchange.ACH-0002...,Avana19Q2_readcount.batch0.foldchange.ACH-0002...,0
ACH-000118,Avana19Q2_readcount.batch0.foldchange,"13,14,15,16",Avana19Q2_readcount.batch0.foldchange.ACH-0001...,Avana19Q2_readcount.batch0.foldchange.ACH-0001...,0
ACH-000791,Avana19Q2_readcount.batch0.foldchange,"17,18,19,20",Avana19Q2_readcount.batch0.foldchange.ACH-0007...,Avana19Q2_readcount.batch0.foldchange.ACH-0007...,0
...,...,...,...,...,...
ACH-001715,Avana19Q2_readcount.batch3.foldchange,"740,741",Avana19Q2_readcount.batch3.foldchange.ACH-0017...,Avana19Q2_readcount.batch3.foldchange.ACH-0017...,3
ACH-000682,Avana19Q2_readcount.batch3.foldchange,"742,743",Avana19Q2_readcount.batch3.foldchange.ACH-0006...,Avana19Q2_readcount.batch3.foldchange.ACH-0006...,3
ACH-000980,Avana19Q2_readcount.batch3.foldchange,"744,745",Avana19Q2_readcount.batch3.foldchange.ACH-0009...,Avana19Q2_readcount.batch3.foldchange.ACH-0009...,3
ACH-001521,Avana19Q2_readcount.batch3.foldchange,746,Avana19Q2_readcount.batch3.foldchange.ACH-0015...,Avana19Q2_readcount.batch3.foldchange.ACH-0015...,3


In [29]:
## parsing Human GENCODE for exon info

gene2transcript = dict()
transcript2exon = dict()
transcript2type = dict()
transcript2CDS = dict()
transcriptlength = dict()
gene2location = dict()
with open('gencode.v28lift37.annotation.gtf','r') as fp:
    for line in fp:
        if line[0] == '#':
            continue
        linearray = line.rstrip().split("\t")
        #print linearray  # ['chr1', 'HAVANA', 'gene', '3073253', '3074322', '.', '+', '.', 'gene_id "ENSMUSG00000102693.1"; gene_type "TEC"; gene_name "RP23-271O17.1"; level 2; havana_gene "OTTMUSG00000049935.1";']
        chrom = linearray[0]
        featuretype = linearray[2]
        start = int(linearray[3])
        end = int(linearray[4])
        strand = linearray[6]
        tags = [x.strip().replace('"','').split(' ') for x in linearray[8].split(";")]
        
        if featuretype=='exon':
            gene=""
            transcript=""
            #genetype = ""
            for tag in tags:
                if tag[0] == 'gene_name':#'gene_id':
                    gene=tag[1].split(".")[0]
                elif tag[0] == 'transcript_id':
                    transcript = tag[1]
                elif tag[0] == 'exon_number':
                    exon = int(tag[1])
                if tag[0] == 'tag' and 'appris_principal' in tag[1]:
                    transcript2type[transcript] = tag[1]
                
                #elif tag[0] == 'tag':
            
            if gene not in gene2transcript:
                gene2transcript[gene]=set()
            gene2transcript[gene].add(transcript)
            if transcript not in transcript2exon:
                transcript2exon[transcript] = dict()
                transcriptlength[transcript]=0
            transcript2exon[transcript][exon] = (chrom,start,end)
            transcriptlength[transcript] += abs(end-start)+1
        
        
        #break

In [30]:
# map gene to longest transcript
gene2longest = dict()
for gene in gene2transcript:
    maxt = list(gene2transcript[gene])[0]
    maxl = transcriptlength[maxt]
    for transcript in gene2transcript[gene]:
        if transcript in transcript2type:
            if maxl < transcriptlength[transcript]: 
                maxt = transcript
                maxl = transcriptlength[transcript]
    gene2longest[gene] = maxt

In [34]:
def check_exon_num(gene,chrom,location):
    result = 0
    if gene not in gene2longest:
        #print (gene)
        return 0
    transcript = gene2longest[gene]
    if chrom != transcript2exon[transcript][1][0]:
        #print (gene, "Chromosome is different %s != Ref = %s"%(chrom,transcript2exon[transcript][1][0]))
        return 0
    
    for exon_num in transcript2exon[transcript]:
        if transcript2exon[transcript][exon_num][1] <=location and location < transcript2exon[transcript][exon_num][2]:
            result = exon_num
    '''if result == 0:
        print gene, location'''
    return result

#Generate library map for CRISPRcleanR format
guidegenemap_filtered = guidegenemap.loc[intersect1d(list(targetsgrna.keys()),readcount.index)]
guidegenemap_CRISPRcleanR = pd.DataFrame(index = intersect1d(list(targetsgrna.keys()),readcount.index),columns=['CODE','GENES','EXONE','CHRM','STRAND','STARTpos','ENDpos'])

exones = dict()
chroms = dict()
strands=dict()
starts = dict()
ends=dict()
for i in range(len(guidegenemap_filtered.index)):
    genome_align = guidegenemap_filtered.iloc[i]['genome_alignment']
    sgrna = guidegenemap_filtered.index[i]
    temp = genome_align.split("_")
    chrom = temp[0].replace('chr','')
    pos = temp[1]
    strand = temp[2]
    if strand =='-':
        posstart = int(pos)
        posend = int(pos) + 19
    if strand =='+':
        posstart = int(pos) - 19
        posend = int(pos)
    exones[sgrna] = 'ex' + str(check_exon_num(targetsgrna[sgrna],temp[0],int(pos)))
    chroms[sgrna] = chrom
    strands[sgrna] = strand
    starts[sgrna] = posstart
    ends[sgrna] = posend
    #guidegenemap_CRISPRcleanR.loc[sgrna] = [targetsgrna[sgrna],targetsgrna[sgrna],'-',chrom,strand,posstart,posend]

    
guidegenemap_CRISPRcleanR['CODE'] = pd.Series(targetsgrna)
guidegenemap_CRISPRcleanR['GENES'] = pd.Series(targetsgrna)
guidegenemap_CRISPRcleanR['EXONE'] = pd.Series(exones)
guidegenemap_CRISPRcleanR['CHRM'] = pd.Series(chroms)
guidegenemap_CRISPRcleanR['STRAND'] = pd.Series(strands)
guidegenemap_CRISPRcleanR['STARTpos'] = pd.Series(starts)
guidegenemap_CRISPRcleanR['ENDpos'] = pd.Series(starts)
guidegenemap_CRISPRcleanR.index.rename("sgRNA",inplace=True)

In [41]:
guidegenemap_CRISPRcleanR.head(3)

,CODE,GENES,EXONE,CHRM,STRAND,STARTpos,ENDpos
sgRNA,,,,,,,
AAAAAAATCCAGCAATGCAG,SHOC2,SHOC2,ex2,10,+,112724359,112724359
AAAAAACCCGTAGATAGCCT,NDUFA12,NDUFA12,ex1,12,+,95397372,95397372
AAAAAAGAAGAAAAAACCAG,SDAD1,SDAD1,ex10,4,-,76891509,76891509


In [42]:
guidegenemap_CRISPRcleanR.to_csv('Avana_library_filtered_forCRISPRcleanR',sep='\t')